## Imports

In [1]:
import numpy as np
import open3d as o3d
from scipy.optimize import least_squares
from sklearn.cluster import DBSCAN

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Load cloud

In [2]:
pcd = o3d.io.read_point_cloud("filtered2.pcd")
downpcd = pcd.voxel_down_sample(voxel_size=0.005)
points = np.asarray(downpcd.points)
downpcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.01, max_nn=30))
normals = np.asarray(downpcd.normals)

## RANSAC Cone

In [60]:
import numpy as np
from scipy.optimize import least_squares
from sklearn.neighbors import NearestNeighbors

def calculate_density(points, radius=0.1):
    """
    Calculates density of points in the neighborhood.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        radius (float): Radius to consider for density calculation.

    Returns:
        ndarray: Density values for each point.
    """
    nbrs = NearestNeighbors(radius=radius).fit(points)
    densities = np.array([len(nbrs.radius_neighbors([point], radius=radius, return_distance=False)[0]) for point in points])
    return densities

def calculate_cone_score(normals, expected_normals):
    """
    Calculates a score for each point based on how well its normal aligns with the expected cone normal.

    Parameters:
        normals (ndarray): Nx3 array of normal vectors.
        expected_normals (ndarray): Nx3 array of expected normal vectors.

    Returns:
        ndarray: Score values for each point.
    """
    # Normalize expected normals and input normals
    expected_normals = expected_normals / np.linalg.norm(expected_normals, axis=1, keepdims=True)
    normals = normals / np.linalg.norm(normals, axis=1, keepdims=True)

    # Calculate the cosine similarity (dot product) between normals and expected normals
    normal_alignment = np.einsum('ij,ij->i', normals, expected_normals)
    
    # Apply a scoring function that emphasizes strong alignment
    scores = np.exp(20 * normal_alignment)  # Adjust scaling factor for stricter alignment
    return scores

def calculate_expected_cone_normals(points, params):
    """
    Calculates the expected normal vectors on the cone surface for each point.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        params (ndarray): Cone parameters [x0, y0, z0, a, b, c, theta].

    Returns:
        ndarray: Expected normal vectors for each point.
    """
    x0, y0, z0, a, b, c, theta = params
    cone_vertex = np.array([x0, y0, z0])
    direction_vector = np.array([a, b, c])
    direction_vector /= np.linalg.norm(direction_vector)

    # Vector from the cone vertex to each point
    vector_to_point = points - cone_vertex
    # Projection of the vector onto the cone's axis
    projection_length = np.dot(vector_to_point, direction_vector)
    projection = projection_length[:, np.newaxis] * direction_vector
    # Tangent vector pointing outward from the cone's surface
    tangent_vector = vector_to_point - projection
    tangent_vector /= np.linalg.norm(tangent_vector, axis=1, keepdims=True)

    # Expected normal vectors that are perpendicular to the cone's surface at the given point
    expected_normals = np.cross(tangent_vector, direction_vector)
    expected_normals /= np.linalg.norm(expected_normals, axis=1, keepdims=True)

    # Adjust normals to match the cone surface angle
    surface_normals = np.sin(theta) * direction_vector + np.cos(theta) * expected_normals
    surface_normals /= np.linalg.norm(surface_normals, axis=1, keepdims=True)

    return surface_normals

def fit_cone_with_fixed_height(points, h_min, h_max, threshold, theta_min=np.deg2rad(5), theta_max=np.deg2rad(80)):
    def residuals(params, points):
        x0, y0, z0, a, b, c, theta = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)

        # Vector from the cone vertex to each point
        vector_to_point = points - np.array([x0, y0, z0])
        # Projection of the vector onto the cone's axis
        projection_length = np.dot(vector_to_point, direction_vector)
        # Projected vector on the cone's axis
        projection = projection_length[:, np.newaxis] * direction_vector
        # Perpendicular distance from the point to the axis of the cone
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        # Expected distance based on the cone's angle
        expected_distance = np.tan(theta) * projection_length

        # Check if points are outside the height range
        out_of_bounds = (projection_length < h_min) | (projection_length > h_max)
        residuals = perpendicular_distance - expected_distance

        # Apply a large penalty for points outside the valid height range
        residuals[out_of_bounds] = 10 * threshold  # This penalty value can be tuned

        return residuals

    # Initial parameter guesses
    x0, y0, z0 = np.mean(points, axis=0)  # Cone vertex near the data center
    direction = np.array([0, 0, 1])  # Initial direction pointing upwards
    theta0 = np.pi / 6  # Initial guess for opening angle (30 degrees)

    # Combine all initial guesses
    initial_guess = [x0, y0, z0, direction[0], direction[1], direction[2], theta0]

    # Set bounds to ensure the angle stays within a realistic range
    bounds = ([-np.inf] * 6 + [theta_min], [np.inf] * 6 + [theta_max])

    # Optimize the cone parameters with angle constraints
    result = least_squares(residuals, initial_guess, args=(points,), bounds=bounds)
    
    # Check the direction of the fitted cone
    params = result.x
    a, b, c = params[3:6]
    
    # Flip the cone if it's upside down based on the z-component
    if c < 0:
        params[3:6] = -np.array([a, b, c])
        params[6] = np.pi - params[6]  # Adjust the angle to keep the cone geometry consistent

    return params

def ransac_cone_with_strict_scoring(points, normals, threshold, iterations, h_min, h_max, theta_min=np.deg2rad(5), theta_max=np.deg2rad(80)):
    densities = calculate_density(points)

    best_inliers = []
    best_params = None
    best_score = 0

    for _ in range(iterations):
        # Randomly select a subset of points to fit the cone
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        try:
            params = fit_cone_with_fixed_height(sample, h_min, h_max, threshold, theta_min, theta_max)
        except:
            continue

        # Calculate the expected normals for the cone model
        expected_normals = calculate_expected_cone_normals(points, params)
        scores = calculate_cone_score(normals, expected_normals)
        inliers = []

        # Evaluate all points to count inliers with strict scoring
        for idx, point in enumerate(points):
            x0, y0, z0, a, b, c, theta = params
            direction_vector = np.array([a, b, c])
            direction_vector /= np.linalg.norm(direction_vector)

            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length

            residual = np.abs(perpendicular_distance - expected_distance)

            # Use stricter score function to enforce harsher penalties
            if h_min <= projection_length <= h_max and residual < threshold and scores[idx] > 0.5:
                inliers.append((point, scores[idx] * densities[idx]))

        # Calculate the total score for the inliers
        total_score = sum([score for _, score in inliers])

        # Update best model if the new model has a higher score
        if total_score > best_score or (len(inliers) > len(best_inliers) and total_score > 0.7 * best_score):
            best_inliers = inliers
            best_params = params
            best_score = total_score

    # Extract inlier points from the best inliers
    best_inlier_points = [inlier[0] for inlier in best_inliers]
    return best_params, np.array(best_inlier_points)

In [70]:
import numpy as np
from scipy.optimize import least_squares
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

def calculate_density(points, radius=0.1):
    """
    Calculates density of points in the neighborhood.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        radius (float): Radius to consider for density calculation.

    Returns:
        ndarray: Density values for each point.
    """
    nbrs = NearestNeighbors(radius=radius).fit(points)
    densities = np.array([len(nbrs.radius_neighbors([point], radius=radius, return_distance=False)[0]) for point in points])
    return densities

def calculate_cone_score(normals, expected_normals):
    """
    Calculates a score for each point based on how well its normal aligns with the expected cone normal.

    Parameters:
        normals (ndarray): Nx3 array of normal vectors.
        expected_normals (ndarray): Nx3 array of expected normal vectors.

    Returns:
        ndarray: Score values for each point.
    """
    # Normalize both expected normals and actual normals
    expected_normals = expected_normals / np.linalg.norm(expected_normals, axis=1, keepdims=True)
    normals = normals / np.linalg.norm(normals, axis=1, keepdims=True)

    # Calculate the dot product (cosine similarity) between normals and expected normals
    normal_alignment = np.einsum('ij,ij->i', normals, expected_normals)

    # Calculate the angular deviation in degrees
    angular_deviation = np.degrees(np.arccos(np.clip(normal_alignment, -1.0, 1.0)))

    # Scores: high alignment (low angular deviation) gives a high score
    scores = np.exp(-0.1 * angular_deviation ** 2)  # Adjust the factor for sensitivity control
    return scores

def calculate_expected_cone_normals(points, params):
    """
    Calculates the expected normal vectors on the cone surface for each point.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        params (ndarray): Cone parameters [x0, y0, z0, a, b, c, theta].

    Returns:
        ndarray: Expected normal vectors for each point.
    """
    x0, y0, z0, a, b, c, theta = params
    cone_vertex = np.array([x0, y0, z0])
    direction_vector = np.array([a, b, c])
    direction_vector /= np.linalg.norm(direction_vector)

    # Vector from the cone vertex to each point
    vector_to_point = points - cone_vertex
    # Projection of the vector onto the cone's axis
    projection_length = np.dot(vector_to_point, direction_vector)
    projection = projection_length[:, np.newaxis] * direction_vector
    # Tangent vector pointing outward from the cone's surface
    tangent_vector = vector_to_point - projection
    tangent_vector /= np.linalg.norm(tangent_vector, axis=1, keepdims=True)

    # Calculate the expected normals based on the cone's surface geometry
    expected_normals = tangent_vector * np.cos(theta) + direction_vector * np.sin(theta)
    expected_normals /= np.linalg.norm(expected_normals, axis=1, keepdims=True)

    return expected_normals

def improved_initial_guess(points):
    """
    Generates an improved initial guess for the cone's parameters using PCA and other heuristics.

    Parameters:
        points (ndarray): Nx3 array of 3D points.

    Returns:
        list: Initial guess of [x0, y0, z0, a, b, c, theta].
    """
    # Use PCA to determine the main axis direction
    pca = PCA(n_components=3)
    pca.fit(points)
    direction = pca.components_[0]  # Assume the main axis is the first principal component

    # Use the mean of points as an initial guess for the cone vertex
    x0, y0, z0 = np.mean(points, axis=0)
    theta0 = np.deg2rad(20)  # Start with a reasonable angle guess (20 degrees)

    # Ensure direction points upwards; adjust if needed
    if direction[2] < 0:
        direction = -direction

    return [x0, y0, z0, direction[0], direction[1], direction[2], theta0]

def fit_cone_with_fixed_height(points, h_min, h_max, threshold, theta_min=np.deg2rad(5), theta_max=np.deg2rad(80)):
    def residuals(params, points):
        x0, y0, z0, a, b, c, theta = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)

        # Vector from the cone vertex to each point
        vector_to_point = points - np.array([x0, y0, z0])
        # Projection of the vector onto the cone's axis
        projection_length = np.dot(vector_to_point, direction_vector)
        # Projected vector on the cone's axis
        projection = projection_length[:, np.newaxis] * direction_vector
        # Perpendicular distance from the point to the axis of the cone
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        # Expected distance based on the cone's angle
        expected_distance = np.tan(theta) * projection_length

        # Check if points are outside the height range
        out_of_bounds = (projection_length < h_min) | (projection_length > h_max)
        residuals = perpendicular_distance - expected_distance

        # Apply a large penalty for points outside the valid height range
        residuals[out_of_bounds] = 10 * threshold  # This penalty value can be tuned

        return residuals

    # Improved initial guess based on data
    initial_guess = improved_initial_guess(points)

    # Set bounds to ensure the angle stays within a realistic range
    bounds = ([-np.inf] * 6 + [theta_min], [np.inf] * 6 + [theta_max])

    # Optimize the cone parameters with angle constraints
    result = least_squares(residuals, initial_guess, args=(points,), bounds=bounds)
    
    # Check the direction of the fitted cone
    params = result.x
    a, b, c = params[3:6]
    
    # Flip the cone if it's upside down based on the z-component
    if c < 0:
        params[3:6] = -np.array([a, b, c])
        params[6] = np.pi - params[6]  # Adjust the angle to keep the cone geometry consistent

    return params

def ransac_cone_with_strict_scoring(points, normals, threshold, iterations, h_min, h_max, theta_min=np.deg2rad(5), theta_max=np.deg2rad(80)):
    densities = calculate_density(points)

    best_inliers = []
    best_params = None
    best_score = 0

    for _ in range(iterations):
        # Randomly select a subset of points to fit the cone
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        try:
            params = fit_cone_with_fixed_height(sample, h_min, h_max, threshold, theta_min, theta_max)
        except:
            continue

        # Calculate the expected normals for the cone model
        expected_normals = calculate_expected_cone_normals(points, params)
        scores = calculate_cone_score(normals, expected_normals)
        inliers = []

        # Evaluate all points to count inliers with strict scoring
        for idx, point in enumerate(points):
            x0, y0, z0, a, b, c, theta = params
            direction_vector = np.array([a, b, c])
            direction_vector /= np.linalg.norm(direction_vector)

            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length

            residual = np.abs(perpendicular_distance - expected_distance)

            # Use stricter score function to enforce harsher penalties
            if h_min <= projection_length <= h_max and residual < threshold and scores[idx] > 0.5:
                inliers.append((point, scores[idx] * densities[idx]))

In [101]:
import numpy as np
from scipy.optimize import least_squares
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

def safe_normalize(vectors):
    """
    Safely normalizes vectors to avoid division by zero.

    Parameters:
        vectors (ndarray): Nx3 array of vectors.

    Returns:
        ndarray: Normalized vectors.
    """
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Prevent division by zero
    return vectors / norms

def calculate_density(points, radius=0.1):
    """
    Calculates density of points in the neighborhood.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        radius (float): Radius to consider for density calculation.

    Returns:
        ndarray: Density values for each point.
    """
    nbrs = NearestNeighbors(radius=radius).fit(points)
    densities = np.array([len(nbrs.radius_neighbors([point], radius=radius, return_distance=False)[0]) for point in points])
    return densities

def calculate_cone_score(normals, expected_normals, points, params, alignment_weight=10, proximity_weight=1):
    """
    Calculates a score for each point based on how well its normal aligns with the expected cone normal.

    Parameters:
        normals (ndarray): Nx3 array of normal vectors.
        expected_normals (ndarray): Nx3 array of expected normal vectors.
        points (ndarray): Nx3 array of 3D points.
        params (ndarray): Cone parameters [x0, y0, z0, a, b, c, theta].
        alignment_weight (float): Weight for alignment scoring.
        proximity_weight (float): Weight for proximity scoring.

    Returns:
        ndarray: Score values for each point.
    """
    # Safely normalize both expected normals and actual normals
    expected_normals = safe_normalize(expected_normals)
    normals = safe_normalize(normals)

    # Calculate the dot product (cosine similarity) between normals and expected normals
    normal_alignment = np.einsum('ij,ij->i', normals, expected_normals)
    # Clamp values to avoid invalid inputs to arccos
    normal_alignment = np.clip(normal_alignment, -1.0, 1.0)

    # Alignment score: penalize deviations less strictly to include more points
    alignment_scores = np.exp(-alignment_weight * (1 - normal_alignment) ** 2)

    # Calculate perpendicular distance from points to the cone surface
    x0, y0, z0, a, b, c, theta = params
    cone_vertex = np.array([x0, y0, z0])
    direction_vector = np.array([a, b, c])
    direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

    # Vector from the cone vertex to each point
    vector_to_point = points - cone_vertex
    # Projection of the vector onto the cone's axis
    projection_length = np.dot(vector_to_point, direction_vector)
    projection = projection_length[:, np.newaxis] * direction_vector
    perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
    expected_distance = np.tan(theta) * projection_length

    # Proximity score: prioritize points near the cone surface with a smoother penalty
    proximity_scores = np.exp(-proximity_weight * (perpendicular_distance - expected_distance) ** 2)

    # Combine alignment and proximity scores, allowing some flexibility in both
    combined_scores = alignment_scores * proximity_scores
    return combined_scores

def calculate_expected_cone_normals(points, params):
    """
    Calculates the expected normal vectors on the cone surface for each point.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        params (ndarray): Cone parameters [x0, y0, z0, a, b, c, theta].

    Returns:
        ndarray: Expected normal vectors for each point.
    """
    x0, y0, z0, a, b, c, theta = params
    cone_vertex = np.array([x0, y0, z0])
    direction_vector = np.array([a, b, c])
    direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

    # Vector from the cone vertex to each point
    vector_to_point = points - cone_vertex
    # Projection of the vector onto the cone's axis
    projection_length = np.dot(vector_to_point, direction_vector)
    projection = projection_length[:, np.newaxis] * direction_vector
    # Tangent vector pointing outward from the cone's surface
    tangent_vector = vector_to_point - projection
    tangent_vector = safe_normalize(tangent_vector)

    # Calculate the expected normals based on the cone's surface geometry
    expected_normals = tangent_vector * np.cos(theta) + direction_vector * np.sin(theta)
    expected_normals = safe_normalize(expected_normals)

    return expected_normals

def improved_initial_guess(points):
    """
    Generates an improved initial guess for the cone's parameters using the highest point as the vertex.

    Parameters:
        points (ndarray): Nx3 array of 3D points.

    Returns:
        list: Initial guess of [x0, y0, z0, a, b, c, theta].
    """
    # Start the cone vertex near the highest point in the cloud
    x0, y0, z0 = points[np.argmax(points[:, 2])]  # Set vertex at the highest z-coordinate point

    # Use PCA to determine the main axis direction
    pca = PCA(n_components=3)
    pca.fit(points)
    direction = pca.components_[0]  # Assume the main axis is the first principal component

    # Adjust the direction to point downwards from the top
    if direction[2] > 0:
        direction = -direction

    theta0 = np.deg2rad(20)  # Start with a reasonable angle guess (20 degrees)

    return [x0, y0, z0, direction[0], direction[1], direction[2], theta0]

def fit_cone_with_fixed_height(points, h_min, h_max, threshold, theta_min=np.deg2rad(5), theta_max=np.deg2rad(80)):
    def residuals(params, points):
        x0, y0, z0, a, b, c, theta = params
        direction_vector = np.array([a, b, c])
        direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

        # Vector from the cone vertex to each point
        vector_to_point = points - np.array([x0, y0, z0])
        # Projection of the vector onto the cone's axis
        projection_length = np.dot(vector_to_point, direction_vector)
        # Projected vector on the cone's axis
        projection = projection_length[:, np.newaxis] * direction_vector
        # Perpendicular distance from the point to the axis of the cone
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        # Expected distance based on the cone's angle
        expected_distance = np.tan(theta) * projection_length

        # Check if points are outside the height range
        out_of_bounds = (projection_length < h_min) | (projection_length > h_max)
        residuals = perpendicular_distance - expected_distance

        # Apply a large penalty for points outside the valid height range
        residuals[out_of_bounds] = 10 * threshold  # This penalty value can be tuned

        return residuals

    # Improved initial guess based on data
    initial_guess = improved_initial_guess(points)

    # Set bounds to ensure the angle stays within a realistic range
    bounds = ([-np.inf] * 6 + [theta_min], [np.inf] * 6 + [theta_max])

    # Optimize the cone parameters with angle constraints
    result = least_squares(residuals, initial_guess, args=(points,), bounds=bounds)
    
    # Check the direction of the fitted cone
    params = result.x
    a, b, c = params[3:6]
    
    # Flip the cone if it's upside down based on the z-component
    if c > 0:
        params[3:6] = -np.array([a, b, c])
        params[6] = np.pi - params[6]  # Adjust the angle to keep the cone geometry consistent

    return params

def ransac_cone_with_strict_scoring(points, normals, threshold, iterations, h_min, h_max, theta_min=np.deg2rad(15), theta_max=np.deg2rad(60)):
    densities = calculate_density(points)

    best_inliers = []
    best_params = None
    best_score = 0

    for _ in range(iterations):
        # Randomly select a subset of points to fit the cone
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        try:
            params = fit_cone_with_fixed_height(sample, h_min, h_max, threshold, theta_min, theta_max)
        except:
            continue

        # Calculate the expected normals for the cone model
        expected_normals = calculate_expected_cone_normals(points, params)
        scores = calculate_cone_score(normals, expected_normals, points, params, alignment_weight=5, proximity_weight=0.5)
        inliers = []

        # Evaluate all points to count inliers with balanced scoring
        for idx, point in enumerate(points):
            x0, y0, z0, a, b, c, theta = params
            direction_vector = np.array([a, b, c])
            direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length

            residual = np.abs(perpendicular_distance - expected_distance)

            # Use adjusted score thresholds to encourage broader inlier selection
            if h_min <= projection_length <= h_max and residual < threshold and scores[idx] > 0.2:
                inliers.append((point, scores[idx] * densities[idx]))

        # Calculate the total score for the inliers
        total_score = sum([score for _, score in inliers])

        # Update best model if the new model has a higher score
        if total_score > best_score or (len(inliers) > len(best_inliers) and total_score > 0.7 * best_score):
            best_inliers = inliers
            best_params = params
            best_score = total_score

    # Extract inlier points from the best inliers
    best_inlier_points = [inlier[0] for inlier in best_inliers]
    return best_params, np.array(best_inlier_points)

In [112]:
import numpy as np
from scipy.optimize import least_squares
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

def safe_normalize(vectors):
    """
    Safely normalizes vectors to avoid division by zero.

    Parameters:
        vectors (ndarray): Nx3 array of vectors.

    Returns:
        ndarray: Normalized vectors.
    """
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Prevent division by zero
    return vectors / norms

def calculate_density(points, radius=0.1):
    """
    Calculates density of points in the neighborhood.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        radius (float): Radius to consider for density calculation.

    Returns:
        ndarray: Density values for each point.
    """
    nbrs = NearestNeighbors(radius=radius).fit(points)
    densities = np.array([len(nbrs.radius_neighbors([point], radius=radius, return_distance=False)[0]) for point in points])
    return densities

def calculate_cone_score(normals, expected_normals, points, params, alignment_weight=10, proximity_weight=1):
    """
    Calculates a score for each point based on how well its normal aligns with the expected cone normal.

    Parameters:
        normals (ndarray): Nx3 array of normal vectors.
        expected_normals (ndarray): Nx3 array of expected normal vectors.
        points (ndarray): Nx3 array of 3D points.
        params (ndarray): Cone parameters [x0, y0, z0, a, b, c, theta].
        alignment_weight (float): Weight for alignment scoring.
        proximity_weight (float): Weight for proximity scoring.

    Returns:
        ndarray: Score values for each point.
    """
    # Safely normalize both expected normals and actual normals
    expected_normals = safe_normalize(expected_normals)
    normals = safe_normalize(normals)

    # Calculate the dot product (cosine similarity) between normals and expected normals
    normal_alignment = np.einsum('ij,ij->i', normals, expected_normals)
    # Clamp values to avoid invalid inputs to arccos
    normal_alignment = np.clip(normal_alignment, -1.0, 1.0)

    # Alignment score: penalize deviations less strictly to include more points
    alignment_scores = np.exp(-alignment_weight * (1 - normal_alignment) ** 2)

    # Calculate perpendicular distance from points to the cone surface
    x0, y0, z0, a, b, c, theta = params
    cone_vertex = np.array([x0, y0, z0])
    direction_vector = np.array([a, b, c])
    direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

    # Vector from the cone vertex to each point
    vector_to_point = points - cone_vertex
    # Projection of the vector onto the cone's axis
    projection_length = np.dot(vector_to_point, direction_vector)
    projection = projection_length[:, np.newaxis] * direction_vector
    perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
    expected_distance = np.tan(theta) * projection_length

    # Proximity score: prioritize points near the cone surface with a smoother penalty
    proximity_scores = np.exp(-proximity_weight * (perpendicular_distance - expected_distance) ** 2)

    # Combine alignment and proximity scores, allowing some flexibility in both
    combined_scores = alignment_scores * proximity_scores
    return combined_scores

def calculate_expected_cone_normals(points, params):
    """
    Calculates the expected normal vectors on the cone surface for each point.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        params (ndarray): Cone parameters [x0, y0, z0, a, b, c, theta].

    Returns:
        ndarray: Expected normal vectors for each point.
    """
    x0, y0, z0, a, b, c, theta = params
    cone_vertex = np.array([x0, y0, z0])
    direction_vector = np.array([a, b, c])
    direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

    # Vector from the cone vertex to each point
    vector_to_point = points - cone_vertex
    # Projection of the vector onto the cone's axis
    projection_length = np.dot(vector_to_point, direction_vector)
    projection = projection_length[:, np.newaxis] * direction_vector

    # Calculate the tangent vector along the cone's surface
    tangent_vector = vector_to_point - projection
    tangent_vector = safe_normalize(tangent_vector)

    # The expected normal is perpendicular to the tangent and reflects the cone's angle
    normal_angle = np.arctan(1 / np.tan(theta))  # Angle between tangent and expected normal

    # Calculate the expected normals by rotating the tangent vector
    expected_normals = tangent_vector * np.cos(normal_angle) + direction_vector * np.sin(normal_angle)
    expected_normals = safe_normalize(expected_normals)

    return expected_normals

def improved_initial_guess(points):
    """
    Generates an improved initial guess for the cone's parameters using the highest point as the vertex.

    Parameters:
        points (ndarray): Nx3 array of 3D points.

    Returns:
        list: Initial guess of [x0, y0, z0, a, b, c, theta].
    """
    # Start the cone vertex near the highest point in the cloud
    x0, y0, z0 = points[np.argmax(points[:, 2])]  # Set vertex at the highest z-coordinate point

    # Use PCA to determine the main axis direction
    pca = PCA(n_components=3)
    pca.fit(points)
    direction = pca.components_[0]  # Assume the main axis is the first principal component

    # Adjust the direction to point downwards from the top
    if direction[2] > 0:
        direction = -direction

    theta0 = np.deg2rad(20)  # Start with a reasonable angle guess (20 degrees)

    return [x0, y0, z0, direction[0], direction[1], direction[2], theta0]

def fit_cone_with_fixed_height(points, h_min, h_max, threshold, theta_min=np.deg2rad(5), theta_max=np.deg2rad(80)):
    def residuals(params, points):
        x0, y0, z0, a, b, c, theta = params
        direction_vector = np.array([a, b, c])
        direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

        # Vector from the cone vertex to each point
        vector_to_point = points - np.array([x0, y0, z0])
        # Projection of the vector onto the cone's axis
        projection_length = np.dot(vector_to_point, direction_vector)
        # Projected vector on the cone's axis
        projection = projection_length[:, np.newaxis] * direction_vector
        # Perpendicular distance from the point to the axis of the cone
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        # Expected distance based on the cone's angle
        expected_distance = np.tan(theta) * projection_length

        # Check if points are outside the height range
        out_of_bounds = (projection_length < h_min) | (projection_length > h_max)
        residuals = perpendicular_distance - expected_distance

        # Apply a large penalty for points outside the valid height range
        residuals[out_of_bounds] = 10 * threshold  # This penalty value can be tuned

        return residuals

    # Improved initial guess based on data
    initial_guess = improved_initial_guess(points)

    # Set bounds to ensure the angle stays within a realistic range
    bounds = ([-np.inf] * 6 + [theta_min], [np.inf] * 6 + [theta_max])

    # Optimize the cone parameters with angle constraints
    result = least_squares(residuals, initial_guess, args=(points,), bounds=bounds)
    
    # Check the direction of the fitted cone
    params = result.x
    a, b, c = params[3:6]
    
    # Flip the cone if it's upside down based on the z-component
    if c > 0:
        params[3:6] = -np.array([a, b, c])
        params[6] = np.pi - params[6]  # Adjust the angle to keep the cone geometry consistent

    return params

def ransac_cone_with_strict_scoring(points, normals, threshold, iterations, h_min, h_max, theta_min=np.deg2rad(5), theta_max=np.deg2rad(80)):
    densities = calculate_density(points)

    best_inliers = []
    best_params = None
    best_score = 0

    for _ in range(iterations):
        # Randomly select a subset of points to fit the cone
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        try:
            params = fit_cone_with_fixed_height(sample, h_min, h_max, threshold, theta_min, theta_max)
        except:
            continue

        # Calculate the expected normals for the cone model
        expected_normals = calculate_expected_cone_normals(points, params)
        scores = calculate_cone_score(normals, expected_normals, points, params, alignment_weight=5)
        inliers = []

        # Evaluate all points to count inliers with balanced scoring
        for idx, point in enumerate(points):
            x0, y0, z0, a, b, c, theta = params
            direction_vector = np.array([a, b, c])
            direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length

            residual = np.abs(perpendicular_distance - expected_distance)

            # Use adjusted score thresholds to encourage broader inlier selection
            if h_min <= projection_length <= h_max and residual < threshold and scores[idx] > 0.2:
                inliers.append((point, scores[idx] * densities[idx]))

        # Calculate the total score for the inliers
        total_score = sum([score for _, score in inliers])

        # Update best model if the new model has a higher score
        if total_score > best_score or (len(inliers) > len(best_inliers) and total_score > 0.7 * best_score):
            best_inliers = inliers
            best_params = params
            best_score = total_score

    # Extract inlier points from the best inliers
    best_inlier_points = [inlier[0] for inlier in best_inliers]
    return best_params, np.array(best_inlier_points)

In [122]:
import numpy as np
from scipy.optimize import least_squares
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

def safe_normalize(vectors):
    """
    Safely normalizes vectors to avoid division by zero.

    Parameters:
        vectors (ndarray): Nx3 array of vectors.

    Returns:
        ndarray: Normalized vectors.
    """
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Prevent division by zero
    return vectors / norms

def calculate_density(points, radius=0.1):
    """
    Calculates density of points in the neighborhood.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        radius (float): Radius to consider for density calculation.

    Returns:
        ndarray: Density values for each point.
    """
    nbrs = NearestNeighbors(radius=radius).fit(points)
    densities = np.array([len(nbrs.radius_neighbors([point], radius=radius, return_distance=False)[0]) for point in points])
    return densities

def calculate_distance_based_score(points, params, height_weight=0.5, distance_weight=1.0):
    """
    Calculates a score for each point based on the perpendicular distance to the cone surface
    and adapts the weighting based on the point's height.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        params (ndarray): Cone parameters [x0, y0, z0, a, b, c, theta].
        height_weight (float): Weighting factor for height influence.
        distance_weight (float): Weighting factor for perpendicular distance influence.

    Returns:
        ndarray: Score values for each point.
    """
    x0, y0, z0, a, b, c, theta = params
    cone_vertex = np.array([x0, y0, z0])
    direction_vector = np.array([a, b, c])
    direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

    # Vector from the cone vertex to each point
    vector_to_point = points - cone_vertex
    # Projection of the vector onto the cone's axis
    projection_length = np.dot(vector_to_point, direction_vector)
    projection = projection_length[:, np.newaxis] * direction_vector

    # Calculate the perpendicular distance from the point to the cone surface
    perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
    expected_distance = np.tan(theta) * projection_length

    # Height weighting: higher points closer to the apex get more influence
    height_scores = np.exp(-height_weight * np.abs(projection_length - np.max(projection_length)))

    # Distance-based scoring: penalize deviations from the expected distance to the cone surface
    distance_scores = np.exp(-distance_weight * (perpendicular_distance - expected_distance) ** 2)

    # Combined score: prioritize distance match but adjust based on height
    combined_scores = distance_scores * height_scores
    return combined_scores

def improved_initial_guess(points):
    """
    Generates an improved initial guess for the cone's parameters using the highest point as the vertex.

    Parameters:
        points (ndarray): Nx3 array of 3D points.

    Returns:
        list: Initial guess of [x0, y0, z0, a, b, c, theta].
    """
    # Start the cone vertex near the highest point in the cloud
    x0, y0, z0 = points[np.argmax(points[:, 2])]  # Set vertex at the highest z-coordinate point

    # Use PCA to determine the main axis direction
    pca = PCA(n_components=3)
    pca.fit(points)
    direction = pca.components_[0]  # Assume the main axis is the first principal component

    # Adjust the direction to point downwards from the top
    if direction[2] > 0:
        direction = -direction

    theta0 = np.deg2rad(20)  # Start with a reasonable angle guess (20 degrees)

    return [x0, y0, z0, direction[0], direction[1], direction[2], theta0]

def fit_cone_with_fixed_height(points, h_min, h_max, threshold, theta_min=np.deg2rad(5), theta_max=np.deg2rad(80)):
    def residuals(params, points):
        x0, y0, z0, a, b, c, theta = params
        direction_vector = np.array([a, b, c])
        direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

        # Vector from the cone vertex to each point
        vector_to_point = points - np.array([x0, y0, z0])
        # Projection of the vector onto the cone's axis
        projection_length = np.dot(vector_to_point, direction_vector)
        # Projected vector on the cone's axis
        projection = projection_length[:, np.newaxis] * direction_vector
        # Perpendicular distance from the point to the axis of the cone
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        # Expected distance based on the cone's angle
        expected_distance = np.tan(theta) * projection_length

        # Check if points are outside the height range
        out_of_bounds = (projection_length < h_min) | (projection_length > h_max)
        residuals = perpendicular_distance - expected_distance

        # Apply a large penalty for points outside the valid height range
        residuals[out_of_bounds] = 10 * threshold  # This penalty value can be tuned

        return residuals

    # Improved initial guess based on data
    initial_guess = improved_initial_guess(points)

    # Set bounds to ensure the angle stays within a realistic range
    bounds = ([-np.inf] * 6 + [theta_min], [np.inf] * 6 + [theta_max])

    # Optimize the cone parameters with angle constraints
    result = least_squares(residuals, initial_guess, args=(points,), bounds=bounds)
    
    # Check the direction of the fitted cone
    params = result.x
    a, b, c = params[3:6]
    
    # Flip the cone if it's upside down based on the z-component
    if c > 0:
        params[3:6] = -np.array([a, b, c])
        params[6] = np.pi - params[6]  # Adjust the angle to keep the cone geometry consistent

    return params

def ransac_cone_with_distance_based_scoring(points, threshold, iterations, h_min, h_max, theta_min=np.deg2rad(15), theta_max=np.deg2rad(80)):
    densities = calculate_density(points)

    best_inliers = []
    best_params = None
    best_score = 0

    for _ in range(iterations):
        # Randomly select a subset of points to fit the cone
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        try:
            params = fit_cone_with_fixed_height(sample, h_min, h_max, threshold, theta_min, theta_max)
        except:
            continue

        # Calculate the distance-based scores for the cone model
        scores = calculate_distance_based_score(points, params, height_weight=0.5, distance_weight=1.0)
        inliers = []

        # Evaluate all points to count inliers with balanced scoring
        for idx, point in enumerate(points):
            x0, y0, z0, a, b, c, theta = params
            direction_vector = np.array([a, b, c])
            direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length

            residual = np.abs(perpendicular_distance - expected_distance)

            # Use adjusted score thresholds to encourage broader inlier selection
            if h_min <= projection_length <= h_max and residual < threshold and scores[idx] > 0.2:
                inliers.append((point, scores[idx] * densities[idx]))

        # Calculate the total score for the inliers
        total_score = sum([score for _, score in inliers])

        # Update best model if the new model has a higher score
        if total_score > best_score or (len(inliers) > len(best_inliers) and total_score > 0.7 * best_score):
            best_inliers = inliers
            best_params = params
            best_score = total_score

    # Extract inlier points from the best inliers
    best_inlier_points = [inlier[0] for inlier in best_inliers]
    return best_params, np.array(best_inlier_points)

In [3]:
import numpy as np
from scipy.optimize import least_squares
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

def safe_normalize(vectors):
    """
    Safely normalizes vectors to avoid division by zero.

    Parameters:
        vectors (ndarray): Nx3 array of vectors.

    Returns:
        ndarray: Normalized vectors.
    """
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Prevent division by zero
    return vectors / norms

def calculate_density(points, radius=0.1):
    """
    Calculates density of points in the neighborhood.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        radius (float): Radius to consider for density calculation.

    Returns:
        ndarray: Density values for each point.
    """
    nbrs = NearestNeighbors(radius=radius).fit(points)
    densities = np.array([len(nbrs.radius_neighbors([point], radius=radius, return_distance=False)[0]) for point in points])
    return densities

def calculate_geometric_consistency_score(points, params, radial_weight=1.0, height_weight=0.5):
    """
    Calculates a score for each point based on geometric consistency with the cone shape.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        params (ndarray): Cone parameters [x0, y0, z0, a, b, c, theta].
        radial_weight (float): Weight for radial consistency scoring.
        height_weight (float): Weight for height consistency scoring.

    Returns:
        ndarray: Score values for each point.
    """
    x0, y0, z0, a, b, c, theta = params
    cone_vertex = np.array([x0, y0, z0])
    direction_vector = np.array([a, b, c])
    direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

    # Vector from the cone vertex to each point
    vector_to_point = points - cone_vertex
    # Projection of the vector onto the cone's axis
    projection_length = np.dot(vector_to_point, direction_vector)
    projection = projection_length[:, np.newaxis] * direction_vector

    # Calculate the radial distance from the axis of the cone
    radial_distance = np.linalg.norm(vector_to_point - projection, axis=1)
    expected_radial_distance = np.tan(theta) * np.abs(projection_length)

    # Radial consistency: how well the points maintain the expected radial distance
    radial_scores = np.exp(-radial_weight * (radial_distance - expected_radial_distance) ** 2)

    # Height consistency: how well the radial distance increases along the cone height
    max_height = np.max(projection_length)
    height_scores = np.exp(-height_weight * np.abs(projection_length - max_height) / max_height)

    # Combined score to reflect geometric consistency with the cone shape
    combined_scores = radial_scores * height_scores
    return combined_scores

def improved_initial_guess(points):
    """
    Generates an improved initial guess for the cone's parameters using the highest point as the vertex.

    Parameters:
        points (ndarray): Nx3 array of 3D points.

    Returns:
        list: Initial guess of [x0, y0, z0, a, b, c, theta].
    """
    # Start the cone vertex near the highest point in the cloud
    x0, y0, z0 = points[np.argmax(points[:, 2])]  # Set vertex at the highest z-coordinate point

    # Use PCA to determine the main axis direction
    pca = PCA(n_components=3)
    pca.fit(points)
    direction = pca.components_[0]  # Assume the main axis is the first principal component

    # Adjust the direction to point downwards from the top
    if direction[2] > 0:
        direction = -direction

    theta0 = np.deg2rad(20)  # Start with a reasonable angle guess (20 degrees)

    return [x0, y0, z0, direction[0], direction[1], direction[2], theta0]

def fit_cone_with_fixed_height(points, h_min, h_max, threshold, theta_min=np.deg2rad(5), theta_max=np.deg2rad(80)):
    def residuals(params, points):
        x0, y0, z0, a, b, c, theta = params
        direction_vector = np.array([a, b, c])
        direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

        # Vector from the cone vertex to each point
        vector_to_point = points - np.array([x0, y0, z0])
        # Projection of the vector onto the cone's axis
        projection_length = np.dot(vector_to_point, direction_vector)
        # Projected vector on the cone's axis
        projection = projection_length[:, np.newaxis] * direction_vector
        # Perpendicular distance from the point to the axis of the cone
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        # Expected distance based on the cone's angle
        expected_distance = np.tan(theta) * projection_length

        # Check if points are outside the height range
        out_of_bounds = (projection_length < h_min) | (projection_length > h_max)
        residuals = perpendicular_distance - expected_distance

        # Apply a large penalty for points outside the valid height range
        residuals[out_of_bounds] = 10 * threshold  # This penalty value can be tuned

        return residuals

    # Improved initial guess based on data
    initial_guess = improved_initial_guess(points)

    # Set bounds to ensure the angle stays within a realistic range
    bounds = ([-np.inf] * 6 + [theta_min], [np.inf] * 6 + [theta_max])

    # Optimize the cone parameters with angle constraints
    result = least_squares(residuals, initial_guess, args=(points,), bounds=bounds)
    
    # Check the direction of the fitted cone
    params = result.x
    a, b, c = params[3:6]
    
    # Flip the cone if it's upside down based on the z-component
    if c > 0:
        params[3:6] = -np.array([a, b, c])
        params[6] = np.pi - params[6]  # Adjust the angle to keep the cone geometry consistent

    return params

def ransac_cone_with_geometric_consistency_scoring(points, threshold, iterations, h_min, h_max, theta_min=np.deg2rad(5), theta_max=np.deg2rad(80)):
    densities = calculate_density(points)

    best_inliers = []
    best_params = None
    best_score = 0

    for _ in range(iterations):
        # Randomly select a subset of points to fit the cone
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        try:
            params = fit_cone_with_fixed_height(sample, h_min, h_max, threshold, theta_min, theta_max)
        except:
            continue

        # Calculate the geometric consistency scores for the cone model
        scores = calculate_geometric_consistency_score(points, params, radial_weight=1.0, height_weight=0.5)
        inliers = []

        # Evaluate all points to count inliers with geometric consistency scoring
        for idx, point in enumerate(points):
            x0, y0, z0, a, b, c, theta = params
            direction_vector = np.array([a, b, c])
            direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length

            residual = np.abs(perpendicular_distance - expected_distance)

            # Use adjusted score thresholds to encourage broader inlier selection
            if h_min <= projection_length <= h_max and residual < threshold and scores[idx] > 0.2:
                inliers.append((point, scores[idx] * densities[idx]))

        # Calculate the total score for the inliers
        total_score = sum([score for _, score in inliers])

        # Update best model if the new model has a higher score
        if total_score > best_score or (len(inliers) > len(best_inliers) and total_score > 0.7 * best_score):
            best_inliers = inliers
            best_params = params
            best_score = total_score

    # Extract inlier points from the best inliers
    best_inlier_points = [inlier[0] for inlier in best_inliers]
    return best_params, np.array(best_inlier_points)

In [42]:
import numpy as np
from scipy.optimize import least_squares
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

def safe_normalize(vectors):
    """
    Safely normalizes vectors to avoid division by zero.

    Parameters:
        vectors (ndarray): Nx3 array of vectors.

    Returns:
        ndarray: Normalized vectors.
    """
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Prevent division by zero
    return vectors / norms

def calculate_density(points, radius=0.1):
    """
    Calculates density of points in the neighborhood.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        radius (float): Radius to consider for density calculation.

    Returns:
        ndarray: Density values for each point.
    """
    nbrs = NearestNeighbors(radius=radius).fit(points)
    densities = np.array([len(nbrs.radius_neighbors([point], radius=radius, return_distance=False)[0]) for point in points])
    return densities

def calculate_expected_normals(points, params):
    """
    Calculates the expected normals on the cone surface for each point.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        params (ndarray): Cone parameters [x0, y0, z0, a, b, c, theta].

    Returns:
        ndarray: Expected normal vectors for each point.
    """
    x0, y0, z0, a, b, c, theta = params
    cone_vertex = np.array([x0, y0, z0])
    direction_vector = np.array([a, b, c])
    direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

    # Vector from the cone vertex to each point
    vector_to_point = points - cone_vertex
    # Projection of the vector onto the cone's axis
    projection_length = np.dot(vector_to_point, direction_vector)
    projection = projection_length[:, np.newaxis] * direction_vector

    # Calculate the tangent vector along the cone's surface
    tangent_vector = vector_to_point - projection
    tangent_vector = safe_normalize(tangent_vector)

    # Expected normal: combine tangent and direction vectors
    normal_angle = np.arctan(1 / np.tan(theta))  # Angle between tangent and expected normal
    expected_normals = tangent_vector * np.cos(normal_angle) + direction_vector * np.sin(normal_angle)
    expected_normals = safe_normalize(expected_normals)

    return expected_normals

def calculate_residuals_and_scores(points, normals, params, distance_weight=1.0, angle_weight=1.0, normal_weight=1.0):
    """
    Calculates residuals and scores for each point based on distance, angular, and normal consistency.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        normals (ndarray): Nx3 array of normal vectors.
        params (ndarray): Cone parameters [x0, y0, z0, a, b, c, theta].
        distance_weight (float): Weight for distance scoring.
        angle_weight (float): Weight for angular consistency scoring.
        normal_weight (float): Weight for normal alignment scoring.

    Returns:
        ndarray: Residuals and combined scores for each point.
    """
    x0, y0, z0, a, b, c, theta = params
    cone_vertex = np.array([x0, y0, z0])
    direction_vector = np.array([a, b, c])
    direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

    # Vector from the cone vertex to each point
    vector_to_point = points - cone_vertex
    # Projection of the vector onto the cone's axis
    projection_length = np.dot(vector_to_point, direction_vector)
    projection = projection_length[:, np.newaxis] * direction_vector

    # Calculate radial distance and expected distance based on the cone's angle
    radial_distance = np.linalg.norm(vector_to_point - projection, axis=1)
    expected_radial_distance = np.tan(theta) * np.abs(projection_length)

    # Residuals based on distance to the expected cone surface
    residuals = np.abs(radial_distance - expected_radial_distance)

    # Angular deviation of points relative to the cone's expected slope
    polar_angle = np.arccos(np.clip(np.dot(vector_to_point, direction_vector) / np.linalg.norm(vector_to_point, axis=1), -1.0, 1.0))
    angle_deviation = np.abs(polar_angle - theta)

    # Calculate expected normals on the cone surface
    expected_normals = calculate_expected_normals(points, params)

    # Calculate normal alignment scores
    normals = safe_normalize(normals)
    normal_alignment = np.einsum('ij,ij->i', normals, expected_normals)
    normal_alignment = np.clip(normal_alignment, -1.0, 1.0)
    normal_scores = np.exp(-normal_weight * (1 - normal_alignment) ** 2)

    # Distance and angular consistency scores
    distance_scores = np.exp(-distance_weight * residuals ** 2)
    angular_scores = np.exp(-angle_weight * angle_deviation ** 2)

    # Combine scores for a comprehensive evaluation
    combined_scores = distance_scores * angular_scores * normal_scores
    return residuals, combined_scores

def improved_initial_guess(points):
    """
    Generates an improved initial guess for the cone's parameters using PCA.

    Parameters:
        points (ndarray): Nx3 array of 3D points.

    Returns:
        list: Initial guess of [x0, y0, z0, a, b, c, theta].
    """
    # Set vertex at the highest z-coordinate point
    x0, y0, z0 = points[np.argmax(points[:, 2])]

    # Use PCA to determine the main axis direction
    pca = PCA(n_components=3)
    pca.fit(points)
    direction = pca.components_[0]

    # Adjust the direction to point downwards
    if direction[2] > 0:
        direction = -direction

    theta0 = np.deg2rad(20)  # Initial angle guess
    return [x0, y0, z0, direction[0], direction[1], direction[2], theta0]

def fit_cone(points, h_min, h_max, threshold, theta_min=np.deg2rad(5), theta_max=np.deg2rad(80)):
    """
    Fits a cone model to the points using least squares with enforced height constraints.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        h_min (float): Minimum height constraint.
        h_max (float): Maximum height constraint.
        threshold (float): Distance threshold for inlier determination.

    Returns:
        ndarray: Best-fit cone parameters.
    """
    def residuals(params, points):
        x0, y0, z0, a, b, c, theta = params
        direction_vector = np.array([a, b, c])
        direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

        # Vector from the cone vertex to each point
        vector_to_point = points - np.array([x0, y0, z0])
        # Projection of the vector onto the cone's axis
        projection_length = np.dot(vector_to_point, direction_vector)
        projection = projection_length[:, np.newaxis] * direction_vector

        # Perpendicular distance from the point to the cone axis
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        # Expected distance based on the cone's angle
        expected_distance = np.tan(theta) * np.abs(projection_length)

        # Check if points are outside the height constraints
        out_of_bounds = (projection_length < h_min) | (projection_length > h_max)

        # Apply a larger penalty to residuals for points outside the height range
        penalty_factor = 100  # Increase this factor to enforce height constraints more strongly
        residuals = perpendicular_distance - expected_distance
        residuals[out_of_bounds] *= penalty_factor

        return residuals

    # Use an improved initial guess
    initial_guess = improved_initial_guess(points)
    bounds = ([-np.inf] * 6 + [theta_min], [np.inf] * 6 + [theta_max])

    # Optimize cone parameters
    try:
        result = least_squares(residuals, initial_guess, args=(points,), bounds=bounds)
        params = result.x

        # Ensure proper orientation
        a, b, c = params[3:6]

        # Flip the cone if it's upside down based on the z-component
        if c > 0:
            params[3:6] = -np.array([a, b, c])
            params[6] = np.pi - params[6]
    except Exception as e:
        print(f"Error during cone fitting: {e}")
        raise

    return params


def ransac_cone_with_normal_consistency(points, normals, threshold, iterations, h_min, h_max, theta_min=np.deg2rad(15), theta_max=np.deg2rad(60)):
    densities = calculate_density(points)

    best_inliers = []
    best_params = None
    best_score = 0

    for iteration in range(iterations):
        # Randomly select a subset of points to fit the cone
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        try:
            params = fit_cone(sample, h_min, h_max, threshold, theta_min, theta_max)
        except Exception as e:
            print(f"Iteration {iteration}: Cone fitting failed with error: {e}")
            continue

        # Calculate residuals and scores with normal consistency
        try:
            residuals, scores = calculate_residuals_and_scores(points, normals, params, distance_weight=1.0, angle_weight=1.0, normal_weight=1.0)
        except Exception as e:
            print(f"Iteration {iteration}: Scoring failed with error: {e}")
            continue

        inliers = []

        # Adaptive thresholding to select inliers
        adaptive_threshold = np.median(residuals) + 1.5 * np.std(residuals)

        # Evaluate all points to count inliers based on combined scoring
        for idx, point in enumerate(points):
            if residuals[idx] < adaptive_threshold and scores[idx] > 0.2:
                inliers.append((point, scores[idx] * densities[idx]))

        # Calculate the total score for the inliers
        total_score = sum([score for _, score in inliers])

        # Update best model if the new model has a higher score
        if total_score > best_score or (len(inliers) > len(best_inliers) and total_score > 0.7 * best_score):
            best_inliers = inliers
            best_params = params
            best_score = total_score

    # Extract inlier points from the best inliers
    best_inlier_points = [inlier[0] for inlier in best_inliers]
    return best_params, np.array(best_inlier_points)

In [115]:
import numpy as np
from scipy.optimize import least_squares
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

def safe_normalize(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Prevent division by zero
    return vectors / norms

def calculate_density(points, radius=0.1):
    nbrs = NearestNeighbors(radius=radius).fit(points)
    densities = np.array([len(nbrs.radius_neighbors([point], radius=radius, return_distance=False)[0]) for point in points])
    return densities

def calculate_expected_normals(params, points):
    """
    Calculates the expected normals on the cone surface for each point.

    Parameters:
        params (ndarray): Cone parameters [x0, y0, z0, a, b, c, theta].
        points (ndarray): Nx3 array of 3D points.

    Returns:
        ndarray: Expected normal vectors for each point.
    """
    x0, y0, z0, a, b, c, theta = params
    cone_vertex = np.array([x0, y0, z0])
    direction_vector = np.array([a, b, c])
    direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

    # Vector from the cone vertex to each point
    vector_to_point = points - cone_vertex
    projection_length = np.dot(vector_to_point, direction_vector)
    projection = projection_length[:, np.newaxis] * direction_vector

    # Tangent vector perpendicular to the cone's axis
    tangent_vector = vector_to_point - projection
    tangent_vector = safe_normalize(tangent_vector)

    # Calculate the expected normal as a combination of tangent and direction vectors
    normal_angle = np.arctan(1 / np.tan(theta))  # Angle between tangent and expected normal
    expected_normals = tangent_vector * np.cos(normal_angle) + direction_vector * np.sin(normal_angle)
    expected_normals = safe_normalize(expected_normals)

    return expected_normals

def calculate_score_based_on_normals(normals, expected_normals):
    """
    Calculates scores based on how well normals align with the expected cone normals.

    Parameters:
        normals (ndarray): Nx3 array of normal vectors.
        expected_normals (ndarray): Nx3 array of expected normal vectors.

    Returns:
        ndarray: Score values for each point based on normal consistency.
    """
    normals = safe_normalize(normals)
    alignment = np.einsum('ij,ij->i', normals, expected_normals)
    alignment = np.clip(alignment, -1.0, 1.0)
    
    # Apply a very steep penalty for misalignment
    scores = np.exp(-100 * (1 - alignment) ** 2)  # Amplified penalty to heavily reward correct alignment
    return scores

def residuals_cone(params, points):
    x0, y0, z0, a, b, c, theta = params
    cone_vertex = np.array([x0, y0, z0])
    direction_vector = np.array([a, b, c])
    direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

    vector_to_point = points - cone_vertex
    projection_length = np.dot(vector_to_point, direction_vector)
    projection = projection_length[:, np.newaxis] * direction_vector
    radial_distance = np.linalg.norm(vector_to_point - projection, axis=1)
    expected_radial_distance = np.tan(theta) * np.abs(projection_length)

    return radial_distance - expected_radial_distance

def fit_cone(points):
    x0, y0, z0 = np.mean(points, axis=0)
    pca = PCA(n_components=3)
    pca.fit(points)
    initial_direction = pca.components_[0]
    if initial_direction[2] > 0:
        initial_direction = -initial_direction
    theta0 = np.deg2rad(20)
    initial_guess = [x0, y0, z0, initial_direction[0], initial_direction[1], initial_direction[2], theta0]

    result = least_squares(residuals_cone, initial_guess, args=(points,))
    return result.x

def ransac_cone_with_normal_matching(points, normals, threshold, iterations):
    densities = calculate_density(points)
    best_inliers = []
    best_params = None
    best_score = 0

    for _ in range(iterations):
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        params = fit_cone(sample)
        x0, y0, z0, a, b, c, theta = params

        # Calculate expected normals based on the cone model
        expected_normals = calculate_expected_normals(params, points)

        # Score each point based on alignment with the expected normals
        scores = calculate_score_based_on_normals(normals, expected_normals)
        inliers = []

        # Calculate residuals to check distance to the cone surface
        vector_to_point = points - np.array([x0, y0, z0])
        projection_length = np.dot(vector_to_point, np.array([a, b, c]))
        projection = projection_length[:, np.newaxis] * np.array([a, b, c])
        radial_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        expected_distance = np.tan(theta) * np.abs(projection_length)
        residuals = np.abs(radial_distance - expected_distance)

        # Strictly filter inliers based on residuals and alignment scores
        for idx, point in enumerate(points):
            if residuals[idx] < threshold and scores[idx] > 0.7:
                inliers.append((point, scores[idx] * densities[idx]))

        total_score = sum([score for _, score in inliers])

        if total_score > best_score or (len(inliers) > len(best_inliers) and total_score > 0.7 * best_score):
            best_inliers = inliers
            best_params = params
            best_score = total_score

    best_inlier_points = [inlier[0] for inlier in best_inliers]
    return best_params, np.array(best_inlier_points)

In [3]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.optimize import least_squares

def calculate_density(points, radius=0.1):
    """
    Calculates density of points in the neighborhood.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        radius (float): Radius to consider for density calculation.

    Returns:
        ndarray: Density values for each point.
    """
    nbrs = NearestNeighbors(radius=radius).fit(points)
    densities = np.array([len(nbrs.radius_neighbors([point], radius=radius, return_distance=False)[0]) for point in points])
    return densities

def calculate_score(normals, apex, axis_direction, alpha, points, max_height):
    """
    Calculates a score for each point based on how well it fits the cone geometry, considering height constraints.

    Parameters:
        normals (ndarray): Nx3 array of normal vectors.
        apex (ndarray): 3-element array representing the apex of the cone.
        axis_direction (ndarray): 3-element array representing the cone axis direction.
        alpha (float): Cone opening angle in radians.
        points (ndarray): Nx3 array of 3D points.
        max_height (float): Maximum allowed height of the cone.

    Returns:
        ndarray: Reward scores for each point.
    """
    axis_direction = axis_direction / np.linalg.norm(axis_direction)  # Normalize axis direction
    normals = normals / np.linalg.norm(normals, axis=1, keepdims=True)  # Normalize normals

    # Vector from apex to each point
    vectors_to_points = points - apex
    projection_length = np.dot(vectors_to_points, axis_direction)

    # Apply height constraint: filter out points beyond max_height
    valid_mask = (projection_length >= 0) & (projection_length <= max_height)
    vectors_to_points = vectors_to_points[valid_mask]
    normals = normals[valid_mask]
    projection_length = projection_length[valid_mask]
    points = points[valid_mask]

    # Compute the tangent direction for the cone surface at each point
    tangent_vectors = vectors_to_points - projection_length[:, np.newaxis] * axis_direction
    tangent_vectors /= np.linalg.norm(tangent_vectors, axis=1, keepdims=True)  # Normalize tangent vectors

    # Calculate the reward score based on how well the normals align with tangent vectors
    normal_alignment = np.abs(np.einsum('ij,ij->i', normals, tangent_vectors))
    alignment_reward = 1 + 5 * np.exp(-10 * (1 - normal_alignment) ** 2)  # High reward for good alignment

    # Calculate expected radii and reward for points matching expected cone shape
    radial_distances = np.linalg.norm(points - apex, axis=1)
    expected_radii = np.tan(alpha) * projection_length
    shape_reward = 1 + 5 * np.exp(-10 * (np.abs(radial_distances - expected_radii) / expected_radii) ** 2)

    # Combine alignment and shape rewards
    combined_scores = alignment_reward * shape_reward
    return combined_scores

def residuals(params, points, max_height):
    """
    Calculate residuals for cone fitting with height constraint.

    Parameters:
        params (list): [x0, y0, z0, a, b, c, alpha] - cone parameters.
        points (ndarray): Nx3 array of points.
        max_height (float): Maximum allowed height of the cone.

    Returns:
        ndarray: Residuals based on cone surface fitting.
    """
    x0, y0, z0, a, b, c, alpha = params
    apex = np.array([x0, y0, z0])
    axis_direction = np.array([a, b, c])
    axis_direction /= np.linalg.norm(axis_direction)  # Normalize axis direction

    vector_to_points = points - apex
    projection_length = np.dot(vector_to_points, axis_direction)

    # Apply height constraint: reject points beyond max_height by setting high residuals
    height_mask = (projection_length >= 0) & (projection_length <= max_height)
    if not np.any(height_mask):
        return np.ones_like(projection_length) * 1e6  # Large residual if no points are valid

    vector_to_points = vector_to_points[height_mask]
    projection_length = projection_length[height_mask]

    projection = projection_length[:, np.newaxis] * axis_direction
    perpendicular_distance = np.linalg.norm(vector_to_points - projection, axis=1)
    calculated_radii = np.tan(alpha) * projection_length  # Radius depends on distance along the axis

    return perpendicular_distance - calculated_radii

def fit_cone(points, min_angle=np.radians(1), max_angle=np.radians(20), max_height=1.0):
    """
    Fits a cone to the given points with constraints for realistic fitting.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        min_angle (float): Minimum allowed angle of the cone in radians.
        max_angle (float): Maximum allowed angle of the cone in radians.
        max_height (float): Maximum allowed height of the cone.

    Returns:
        ndarray: Optimized cone parameters [x0, y0, z0, a, b, c, alpha].
    """
    # Estimate initial apex near the lower end of the point cloud
    x0, y0, z0 = np.median(points, axis=0)
    initial_axis = np.array([0, 0, 1])  # Assume vertical cone initially
    initial_alpha = np.clip(np.radians(5), min_angle, max_angle)  # Constrain initial cone angle
    initial_guess = [x0, y0, z0, *initial_axis, initial_alpha]

    # Set bounds with constraints to guide towards realistic cone fits
    bounds = (
        [-np.inf, -np.inf, -np.inf, -1, -1, -1, min_angle],  # Lower bounds
        [np.inf, np.inf, np.inf, 1, 1, 1, max_angle]        # Upper bounds
    )

    result = least_squares(residuals, initial_guess, args=(points, max_height), bounds=bounds)
    return result.x  # [x0, y0, z0, a, b, c, alpha]

def ransac_cone_with_strict_scoring(points, normals, threshold, iterations, min_angle=np.radians(1), max_angle=np.radians(20), max_height=1.0):
    """
    RANSAC-based cone fitting with a reward-focused scoring system and strict height constraint.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        normals (ndarray): Nx3 array of normal vectors.
        threshold (float): Distance threshold for inlier classification.
        iterations (int): Number of RANSAC iterations.
        min_angle (float): Minimum allowed angle of the cone in radians.
        max_angle (float): Maximum allowed angle of the cone in radians.
        max_height (float): Maximum allowed height of the cone.

    Returns:
        tuple: Best fit cone parameters and inlier points.
    """
    densities = calculate_density(points)
    
    best_inliers = []
    best_params = None
    best_score = 0

    for _ in range(iterations):
        # Randomly select a subset of points to fit the cone
        sample = points[np.random.choice(points.shape[0], 10, replace=False)]
        try:
            params = fit_cone(sample, min_angle, max_angle, max_height)
            x0, y0, z0, a, b, c, alpha = params
        except ValueError:
            continue  # Skip if fitting fails

        # Check angle constraints
        if not (min_angle <= alpha <= max_angle):
            continue

        apex = np.array([x0, y0, z0])
        axis_direction = np.array([a, b, c])
        axis_direction /= np.linalg.norm(axis_direction)  # Normalize direction

        # Compute scores based on a reward-focused scoring mechanism with strict height constraints
        scores = calculate_score(normals, apex, axis_direction, alpha, points, max_height)
        inliers = []

        # Evaluate all points to count inliers based on high reward scores
        for idx, point in enumerate(points):
            vector_to_point = point - apex
            projection_length = np.dot(vector_to_point, axis_direction)
            projection = projection_length * axis_direction

            # Calculate perpendicular distance from the point to the cone surface
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            calculated_radius = np.tan(alpha) * projection_length

            # Measure how far this distance is from the expected radius
            distance = np.abs(perpendicular_distance - calculated_radius)

            # Use high rewards for well-fitting points within the height constraint
            if distance < threshold and scores[idx] > 1.5:  # Reward points with high fit scores
                inliers.append((point, scores[idx] * densities[idx]))

        # Calculate the total score for the inliers
        total_score = sum([score for _, score in inliers])

        # Update best model if the new model has a higher score
        if total_score > best_score or (len(inliers) > len(best_inliers) and total_score > 0.7 * best_score):
            best_inliers = inliers
            best_params = params
            best_score = total_score

    # Extract inlier points from the best inliers
    best_inlier_points = [inlier[0] for inlier in best_inliers]
    return best_params, np.array(best_inlier_points)

In [5]:
import numpy as np
from scipy.optimize import least_squares
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

def safe_normalize(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Prevent division by zero
    return vectors / norms

def calculate_density(points, radius=0.1):
    nbrs = NearestNeighbors(radius=radius).fit(points)
    densities = np.array([len(nbrs.radius_neighbors([point], radius=radius, return_distance=False)[0]) for point in points])
    return densities

def calculate_expected_normals(params, points):
    """
    Calculates the expected normals on the cone surface for each point.

    Parameters:
        params (ndarray): Cone parameters [x0, y0, z0, a, b, c, theta].
        points (ndarray): Nx3 array of 3D points.

    Returns:
        ndarray: Expected normal vectors for each point.
    """
    x0, y0, z0, a, b, c, theta = params
    cone_vertex = np.array([x0, y0, z0])
    direction_vector = np.array([a, b, c])
    direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

    # Vector from the cone vertex to each point
    vector_to_point = points - cone_vertex
    projection_length = np.dot(vector_to_point, direction_vector)
    projection = projection_length[:, np.newaxis] * direction_vector

    # Tangent vector perpendicular to the cone's axis
    tangent_vector = vector_to_point - projection
    tangent_vector = safe_normalize(tangent_vector)

    # Calculate the expected normal as a combination of tangent and direction vectors
    normal_angle = np.arctan(1 / np.tan(theta))  # Angle between tangent and expected normal
    expected_normals = tangent_vector * np.cos(normal_angle) + direction_vector * np.sin(normal_angle)
    expected_normals = safe_normalize(expected_normals)

    return expected_normals

def calculate_adaptive_scores(normals, expected_normals, residuals, densities):
    """
    Calculates adaptive scores for points based on multiple criteria.

    Parameters:
        normals (ndarray): Nx3 array of normal vectors.
        expected_normals (ndarray): Nx3 array of expected normal vectors.
        residuals (ndarray): Residuals based on distance from the cone surface.
        densities (ndarray): Density values for each point.

    Returns:
        ndarray: Combined score values for each point.
    """
    # Normalize normals and calculate alignment with expected normals
    normals = safe_normalize(normals)
    alignment = np.einsum('ij,ij->i', normals, expected_normals)
    alignment = np.clip(alignment, -1.0, 1.0)

    # Score based on normal alignment
    alignment_scores = np.exp(-50 * (1 - alignment) ** 2)

    # Score based on residuals, with a smooth gradient to avoid harsh penalties
    residual_scores = np.exp(-20 * residuals ** 2)

    # Density-aware scoring: prioritizes regions with consistent point distribution
    density_scores = 1 / (1 + np.exp(-densities + np.median(densities)))  # Sigmoid function for density emphasis

    # Combine scores adaptively with weighting to prioritize alignment
    combined_scores = alignment_scores * residual_scores * density_scores

    return combined_scores

def residuals_cone(params, points):
    x0, y0, z0, a, b, c, theta = params
    cone_vertex = np.array([x0, y0, z0])
    direction_vector = np.array([a, b, c])
    direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

    vector_to_point = points - cone_vertex
    projection_length = np.dot(vector_to_point, direction_vector)
    projection = projection_length[:, np.newaxis] * direction_vector
    radial_distance = np.linalg.norm(vector_to_point - projection, axis=1)
    expected_radial_distance = np.tan(theta) * np.abs(projection_length)

    return radial_distance - expected_radial_distance

def fit_cone(points):
    x0, y0, z0 = np.mean(points, axis=0)
    pca = PCA(n_components=3)
    pca.fit(points)
    initial_direction = pca.components_[0]
    if initial_direction[2] > 0:
        initial_direction = -initial_direction
    theta0 = np.deg2rad(20)
    initial_guess = [x0, y0, z0, initial_direction[0], initial_direction[1], initial_direction[2], theta0]

    result = least_squares(residuals_cone, initial_guess, args=(points,))
    return result.x

def ransac_cone_with_adaptive_scoring(points, normals, threshold, iterations):
    densities = calculate_density(points)
    best_inliers = []
    best_params = None
    best_score = 0

    for _ in range(iterations):
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        params = fit_cone(sample)
        x0, y0, z0, a, b, c, theta = params

        # Calculate expected normals based on the cone model
        expected_normals = calculate_expected_normals(params, points)

        # Calculate residuals to check distance to the cone surface
        vector_to_point = points - np.array([x0, y0, z0])
        projection_length = np.dot(vector_to_point, np.array([a, b, c]))
        projection = projection_length[:, np.newaxis] * np.array([a, b, c])
        radial_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        expected_distance = np.tan(theta) * np.abs(projection_length)
        residuals = np.abs(radial_distance - expected_distance)

        # Calculate adaptive scores based on alignment, residuals, and density
        scores = calculate_adaptive_scores(normals, expected_normals, residuals, densities)
        inliers = []

        # Strictly filter inliers based on residuals and adaptive scores
        for idx, point in enumerate(points):
            if residuals[idx] < threshold and scores[idx] > 0.6:
                inliers.append((point, scores[idx] * densities[idx]))

        total_score = sum([score for _, score in inliers])

        if total_score > best_score or (len(inliers) > len(best_inliers) and total_score > 0.7 * best_score):
            best_inliers = inliers
            best_params = params
            best_score = total_score

    best_inlier_points = [inlier[0] for inlier in best_inliers]
    return best_params, np.array(best_inlier_points)


In [10]:
import numpy as np
from scipy.optimize import least_squares
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA

def safe_normalize(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Prevent division by zero
    return vectors / norms

def calculate_density(points, radius=0.1):
    nbrs = NearestNeighbors(radius=radius).fit(points)
    densities = np.array([len(nbrs.radius_neighbors([point], radius=radius, return_distance=False)[0]) for point in points])
    return densities

def residuals_cone(params, points):
    """
    Calculates residuals for fitting a cone.

    Parameters:
        params (ndarray): Cone parameters [x0, y0, z0, a, b, c, theta].
        points (ndarray): Nx3 array of 3D points.

    Returns:
        ndarray: Residual values for each point.
    """
    x0, y0, z0, a, b, c, theta = params
    cone_vertex = np.array([x0, y0, z0])
    direction_vector = np.array([a, b, c])
    direction_vector = safe_normalize(direction_vector[np.newaxis, :])[0]

    vector_to_point = points - cone_vertex
    projection_length = np.dot(vector_to_point, direction_vector)
    projection = projection_length[:, np.newaxis] * direction_vector
    radial_distance = np.linalg.norm(vector_to_point - projection, axis=1)
    expected_radial_distance = np.tan(theta) * np.abs(projection_length)

    residuals = np.abs(radial_distance - expected_radial_distance)
    return residuals

def fit_cone(points):
    """
    Fits a cone to the given points using least squares optimization.

    Parameters:
        points (ndarray): Nx3 array of 3D points.

    Returns:
        ndarray: Fitted cone parameters [x0, y0, z0, a, b, c, theta].
    """
    x0, y0, z0 = np.mean(points, axis=0)
    pca = PCA(n_components=3)
    pca.fit(points)
    initial_direction = pca.components_[0]
    if initial_direction[2] > 0:
        initial_direction = -initial_direction
    theta0 = np.deg2rad(20)  # Use a reasonable initial guess for the cone angle
    initial_guess = [x0, y0, z0, initial_direction[0], initial_direction[1], initial_direction[2], theta0]

    result = least_squares(residuals_cone, initial_guess, args=(points,))
    return result.x

def ransac_cone_basic(points, normals, threshold, iterations):
    """
    Simplified RANSAC-based fitting of a cone with basic scoring.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        normals (ndarray): Nx3 array of normal vectors.
        threshold (float): Threshold for inlier determination.
        iterations (int): Number of iterations for RANSAC.

    Returns:
        tuple: Best-fit cone parameters and inliers.
    """
    best_inliers = []
    best_params = None
    best_score = 0

    for _ in range(iterations):
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        params = fit_cone(sample)
        x0, y0, z0, a, b, c, theta = params

        # Calculate residuals to check distance to the cone surface
        residuals = residuals_cone(params, points)

        # Simplified inlier selection based on residuals
        inliers = [(point, residual) for idx, (point, residual) in enumerate(zip(points, residuals)) if residual < threshold]

        # Sum of inverse residuals for scoring (to prioritize closer fits)
        total_score = sum(1 / (residual + 1e-6) for _, residual in inliers)  # Avoid division by zero

        if total_score > best_score:
            best_inliers = inliers
            best_params = params
            best_score = total_score

    # Extract inlier points from the best inliers
    best_inlier_points = [inlier[0] for inlier in best_inliers]
    return best_params, np.array(best_inlier_points)

In [16]:
import numpy as np

def safe_normalize(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Prevent division by zero
    return vectors / norms

def intersect_planes(p, n):
    """
    Intersect three planes defined by point-normal pairs to find the cone apex.

    Parameters:
        p (ndarray): Array of shape (3, 3) containing three points.
        n (ndarray): Array of shape (3, 3) containing three normals.

    Returns:
        ndarray: Coordinates of the apex if successful; otherwise, None.
    """
    A = n
    b = np.einsum('ij,ij->i', n, p)
    try:
        apex = np.linalg.solve(A, b)
        return apex
    except np.linalg.LinAlgError:
        return None

def fit_cone_from_points_normals(points, normals):
    """
    Fit a cone using three points and their corresponding normals.

    Parameters:
        points (ndarray): Array of shape (3, 3) containing the points.
        normals (ndarray): Array of shape (3, 3) containing the normals.

    Returns:
        tuple: Cone apex, axis direction, and opening angle.
    """
    # Find the apex by intersecting the three planes
    apex = intersect_planes(points, normals)
    if apex is None:
        return None, None, None

    # Calculate the unit vectors from the apex to each point
    unit_vectors = safe_normalize(points - apex)

    # Find the cone's axis by computing the normal of the plane defined by these unit vectors
    v1 = unit_vectors[1] - unit_vectors[0]
    v2 = unit_vectors[2] - unit_vectors[0]
    axis_direction = safe_normalize(np.cross(v1, v2).reshape(1, -1))[0]

    # Calculate the opening angle ω by averaging the angles between point vectors and the axis
    angles = np.arccos(np.clip(np.dot(unit_vectors, axis_direction), -1.0, 1.0))
    opening_angle = np.mean(angles)

    return apex, axis_direction, opening_angle

def ransac_cone_from_article(points, normals, threshold, iterations):
    """
    RANSAC implementation to fit a cone using the method from the article.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        normals (ndarray): Nx3 array of normal vectors.
        threshold (float): Distance threshold for inlier determination.
        iterations (int): Number of iterations for RANSAC.

    Returns:
        tuple: Best-fit cone parameters and inliers.
    """
    best_inliers = []
    best_params = None
    best_score = 0

    for _ in range(iterations):
        # Randomly select three points and their normals
        idx = np.random.choice(points.shape[0], 3, replace=False)
        sample_points = points[idx]
        sample_normals = normals[idx]

        # Fit the cone using the selected points and normals
        apex, axis_direction, opening_angle = fit_cone_from_points_normals(sample_points, sample_normals)

        # Skip this iteration if fitting failed
        if apex is None or axis_direction is None or opening_angle is None:
            continue

        # Calculate residuals to determine inliers
        vector_to_point = points - apex
        projection_length = np.dot(vector_to_point, axis_direction)
        projection = projection_length[:, np.newaxis] * axis_direction
        radial_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        expected_distance = np.tan(opening_angle) * np.abs(projection_length)
        residuals = np.abs(radial_distance - expected_distance)

        # Select inliers based on the residual threshold
        inliers = [(point, residual) for idx, (point, residual) in enumerate(zip(points, residuals)) if residual < threshold]

        # Sum of inverse residuals for scoring (to prioritize closer fits)
        total_score = sum(1 / (residual + 1e-6) for _, residual in inliers)

        if total_score > best_score:
            best_inliers = inliers
            best_params = (apex, axis_direction, opening_angle)
            best_score = total_score

    # Extract inlier points from the best inliers
    best_inlier_points = [inlier[0] for inlier in best_inliers]
    return best_params, np.array(best_inlier_points)

In [27]:
import numpy as np

def safe_normalize(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Prevent division by zero
    return vectors / norms

def intersect_planes(p, n):
    A = n
    b = np.einsum('ij,ij->i', n, p)
    try:
        apex = np.linalg.solve(A, b)
        return apex
    except np.linalg.LinAlgError:
        return None

def fit_cone_from_points_normals(points, normals):
    apex = intersect_planes(points, normals)
    if apex is None:
        return None, None, None

    unit_vectors = safe_normalize(points - apex)
    v1 = unit_vectors[1] - unit_vectors[0]
    v2 = unit_vectors[2] - unit_vectors[0]
    axis_direction = safe_normalize(np.cross(v1, v2).reshape(1, -1))[0]

    angles = np.arccos(np.clip(np.dot(unit_vectors, axis_direction), -1.0, 1.0))
    opening_angle = np.mean(angles)

    return apex, axis_direction, opening_angle

def ransac_cone_with_height_constraint(points, normals, threshold, iterations, h_max):
    best_inliers = []
    best_params = None
    best_score = 0

    for _ in range(iterations):
        idx = np.random.choice(points.shape[0], 3, replace=False)
        sample_points = points[idx]
        sample_normals = normals[idx]

        apex, axis_direction, opening_angle = fit_cone_from_points_normals(sample_points, sample_normals)
        if apex is None or axis_direction is None or opening_angle is None:
            continue

        vector_to_point = points - apex
        projection_length = np.dot(vector_to_point, axis_direction)
        projection = projection_length[:, np.newaxis] * axis_direction
        radial_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        expected_distance = np.tan(opening_angle) * np.abs(projection_length)

        # Apply height constraint to exclude points beyond the maximum height
        height_violations = (projection_length < 0) | (projection_length > h_max)
        residuals = np.abs(radial_distance - expected_distance)

        # Filter points within the height limit and check residuals
        inliers = [(point, residual) for idx, (point, residual) in enumerate(zip(points, residuals))
                   if residual < threshold and not height_violations[idx]]

        total_score = sum(1 / (residual + 1e-6) for _, residual in inliers)

        if total_score > best_score:
            best_inliers = inliers
            best_params = (apex, axis_direction, opening_angle)
            best_score = total_score

    best_inlier_points = [inlier[0] for inlier in best_inliers]
    return best_params, np.array(best_inlier_points)


In [39]:
import numpy as np

def safe_normalize(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Prevent division by zero
    return vectors / norms

def intersect_planes(p, n):
    """
    Intersect three planes defined by point-normal pairs to find the cone apex.

    Parameters:
        p (ndarray): Array of shape (3, 3) containing three points.
        n (ndarray): Array of shape (3, 3) containing three normals.

    Returns:
        ndarray: Coordinates of the apex if successful; otherwise, None.
    """
    A = n
    b = np.einsum('ij,ij->i', n, p)
    try:
        apex = np.linalg.solve(A, b)
        return apex
    except np.linalg.LinAlgError:
        return None

def fit_cone_from_points_normals(points, normals):
    """
    Fit a cone using three points and their corresponding normals.

    Parameters:
        points (ndarray): Array of shape (3, 3) containing the points.
        normals (ndarray): Array of shape (3, 3) containing the normals.

    Returns:
        tuple: Cone apex, axis direction, and opening angle.
    """
    # Find the apex by intersecting the three planes
    apex = intersect_planes(points, normals)
    if apex is None:
        return None, None, None

    # Calculate the unit vectors from the apex to each point
    unit_vectors = safe_normalize(points - apex)

    # Find the cone's axis by computing the normal of the plane defined by these unit vectors
    v1 = unit_vectors[1] - unit_vectors[0]
    v2 = unit_vectors[2] - unit_vectors[0]
    axis_direction = safe_normalize(np.cross(v1, v2).reshape(1, -1))[0]

    # Calculate the opening angle ω by averaging the angles between point vectors and the axis
    angles = np.arccos(np.clip(np.dot(unit_vectors, axis_direction), -1.0, 1.0))
    opening_angle = np.mean(angles)

    return apex, axis_direction, opening_angle

def ransac_cone_with_constraints(points, normals, threshold, iterations, h_max, angle_min, angle_max):
    """
    RANSAC implementation to fit a cone with height and angle constraints.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        normals (ndarray): Nx3 array of normal vectors.
        threshold (float): Distance threshold for inlier determination.
        iterations (int): Number of iterations for RANSAC.
        h_max (float): Maximum height constraint for the cone.
        angle_min (float): Minimum opening angle (in degrees).
        angle_max (float): Maximum opening angle (in degrees).

    Returns:
        tuple: Best-fit cone parameters and inliers.
    """
    # Convert angle constraints from degrees to radians
    angle_min_rad = np.deg2rad(angle_min)
    angle_max_rad = np.deg2rad(angle_max)
    
    best_inliers = []
    best_params = None
    best_score = 0

    for _ in range(iterations):
        # Randomly select three points and their normals
        idx = np.random.choice(points.shape[0], 3, replace=False)
        sample_points = points[idx]
        sample_normals = normals[idx]

        # Fit the cone using the selected points and normals
        apex, axis_direction, opening_angle = fit_cone_from_points_normals(sample_points, sample_normals)

        # Skip this iteration if fitting failed or the opening angle is out of bounds
        if (apex is None or axis_direction is None or opening_angle is None or 
            not (angle_min_rad <= opening_angle <= angle_max_rad)):
            continue

        # Calculate residuals to determine inliers
        vector_to_point = points - apex
        projection_length = np.dot(vector_to_point, axis_direction)
        projection = projection_length[:, np.newaxis] * axis_direction
        radial_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        expected_distance = np.tan(opening_angle) * np.abs(projection_length)

        # Apply height constraint to exclude points beyond the maximum height
        height_violations = (projection_length < 0) | (projection_length > h_max)
        residuals = np.abs(radial_distance - expected_distance)

        # Filter points within the height limit and check residuals
        inliers = [(point, residual) for idx, (point, residual) in enumerate(zip(points, residuals))
                   if residual < threshold and not height_violations[idx]]

        # Sum of inverse residuals for scoring (to prioritize closer fits)
        total_score = sum(1 / (residual + 1e-6) for _, residual in inliers)

        if total_score > best_score:
            best_inliers = inliers
            best_params = (apex, axis_direction, opening_angle)
            best_score = total_score

    # Extract inlier points from the best inliers
    best_inlier_points = [inlier[0] for inlier in best_inliers]
    return best_params, np.array(best_inlier_points)


In [76]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

def safe_normalize(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Prevent division by zero
    return vectors / norms

def calculate_density_gradient(points, apex, axis_direction):
    """
    Calculates the density gradient along the axis of the cone,
    emphasizing the transition from the apex to the base.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        apex (ndarray): 3-element array representing the cone apex.
        axis_direction (ndarray): 3-element array representing the cone's axis direction.

    Returns:
        ndarray: Density gradient values for each point along the cone axis.
    """
    vectors_to_points = points - apex
    projection_length = np.dot(vectors_to_points, axis_direction)

    bin_edges = np.linspace(projection_length.min(), projection_length.max(), num=20)
    density_along_axis = np.histogram(projection_length, bins=bin_edges)[0]

    density_gradient = np.interp(projection_length, bin_edges[:-1] + np.diff(bin_edges) / 2, density_along_axis)

    normalized_gradient = (density_gradient - density_gradient.min()) / (density_gradient.max() - density_gradient.min() + 1e-6)
    return normalized_gradient

def calculate_visible_expected_normals(points, apex, axis_direction, opening_angle):
    """
    Calculates expected normals dynamically based on the visible parts of the cone.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        apex (ndarray): 3-element array representing the cone apex.
        axis_direction (ndarray): 3-element array representing the cone's axis direction.
        opening_angle (float): Opening angle of the cone in radians.

    Returns:
        ndarray: Expected normal vectors for visible points on the cone surface.
    """
    vectors_to_points = points - apex
    projection_length = np.dot(vectors_to_points, axis_direction)
    projection = projection_length[:, np.newaxis] * axis_direction
    
    tangential_vectors = vectors_to_points - projection
    tangential_vectors = safe_normalize(tangential_vectors)
    
    expected_normals = tangential_vectors * np.cos(opening_angle) + axis_direction * np.sin(opening_angle)
    expected_normals = safe_normalize(expected_normals)
    
    return expected_normals

def calculate_scores_for_partial_cone(normals, expected_normals, density_gradient):
    """
    Calculates scores based on the alignment of normals and the density gradient, focusing on visible sections.

    Parameters:
        normals (ndarray): Nx3 array of normal vectors.
        expected_normals (ndarray): Nx3 array of expected normal vectors for visible regions.
        density_gradient (ndarray): Density gradient values indicating the progression from apex to base.

    Returns:
        ndarray: Combined score values for each point based on normal consistency and density gradient.
    """
    normals = safe_normalize(normals)
    alignment = np.einsum('ij,ij->i', normals, expected_normals)
    alignment = np.clip(alignment, -1.0, 1.0)

    alignment_scores = np.exp(-20 * (1 - alignment) ** 2)
    density_scores = 1 / (1 + np.exp(-10 * (density_gradient - 0.5)))

    combined_scores = alignment_scores * density_scores
    return combined_scores

def ransac_cone_for_partial_surfaces(points, normals, threshold, iterations, h_max, angle_min, angle_max):
    """
    RANSAC implementation to fit a cone, adjusting scoring for partial surfaces.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        normals (ndarray): Nx3 array of normal vectors.
        threshold (float): Distance threshold for inlier determination.
        iterations (int): Number of iterations for RANSAC.
        h_max (float): Maximum height constraint for the cone.
        angle_min (float): Minimum opening angle in degrees.
        angle_max (float): Maximum opening angle in degrees.

    Returns:
        tuple: Best-fit cone parameters and inliers.
    """
    angle_min_rad = np.deg2rad(angle_min)
    angle_max_rad = np.deg2rad(angle_max)
    
    best_inliers = []
    best_params = None
    best_score = 0

    for _ in range(iterations):
        idx = np.random.choice(points.shape[0], 3, replace=False)
        sample_points = points[idx]
        sample_normals = normals[idx]

        apex, axis_direction, opening_angle = fit_cone_from_points_normals(sample_points, sample_normals)

        if (apex is None or axis_direction is None or opening_angle is None or 
            not (angle_min_rad <= opening_angle <= angle_max_rad)):
            continue

        vector_to_point = points - apex
        projection_length = np.dot(vector_to_point, axis_direction)
        projection = projection_length[:, np.newaxis] * axis_direction
        radial_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        expected_distance = np.tan(opening_angle) * np.abs(projection_length)

        height_violations = (projection_length < 0) | (projection_length > h_max)
        residuals = np.abs(radial_distance - expected_distance)

        # Calculate expected normals based on only the visible part of the cone
        expected_normals = calculate_visible_expected_normals(points, apex, axis_direction, opening_angle)
        density_gradient = calculate_density_gradient(points, apex, axis_direction)
        
        # Calculate scores focusing on visible sections of the cone
        scores = calculate_scores_for_partial_cone(normals, expected_normals, density_gradient)

        inliers = [(point, score) for idx, (point, residual, score) 
                   in enumerate(zip(points, residuals, scores))
                   if residual < threshold and not height_violations[idx] and score > 0.5]

        total_score = sum(score for _, score in inliers)

        if total_score > best_score or (len(inliers) > len(best_inliers) and total_score > 0.7 * best_score):
            best_inliers = inliers
            best_params = (apex, axis_direction, opening_angle)
            best_score = total_score

    best_inlier_points = [inlier[0] for inlier in best_inliers]
    return best_params, np.array(best_inlier_points)


In [100]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

def safe_normalize(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    norms[norms == 0] = 1  # Prevent division by zero
    return vectors / norms

def calculate_density_gradient(points, apex, axis_direction):
    """
    Calculates the density gradient along the axis of the cone,
    emphasizing the transition from the apex to the base.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        apex (ndarray): 3-element array representing the cone apex.
        axis_direction (ndarray): 3-element array representing the cone's axis direction.

    Returns:
        ndarray: Density gradient values for each point along the cone axis.
    """
    vectors_to_points = points - apex
    projection_length = np.dot(vectors_to_points, axis_direction)

    bin_edges = np.linspace(projection_length.min(), projection_length.max(), num=20)
    density_along_axis = np.histogram(projection_length, bins=bin_edges)[0]

    density_gradient = np.interp(projection_length, bin_edges[:-1] + np.diff(bin_edges) / 2, density_along_axis)

    normalized_gradient = (density_gradient - density_gradient.min()) / (density_gradient.max() - density_gradient.min() + 1e-6)
    return normalized_gradient

def calculate_perpendicularity_scores(normals, points, apex, axis_direction):
    """
    Calculates scores based on how perpendicular the normals are to the model cone's surface.

    Parameters:
        normals (ndarray): Nx3 array of normal vectors.
        points (ndarray): Nx3 array of 3D points.
        apex (ndarray): 3-element array representing the cone apex.
        axis_direction (ndarray): 3-element array representing the cone's axis direction.

    Returns:
        ndarray: Perpendicularity scores for each point.
    """
    vectors_to_points = points - apex
    projection_length = np.dot(vectors_to_points, axis_direction)
    projection = projection_length[:, np.newaxis] * axis_direction
    
    # Calculate the direction perpendicular to the cone axis at each point
    tangential_vectors = vectors_to_points - projection
    tangential_vectors = safe_normalize(tangential_vectors)

    # Calculate the alignment between point normals and tangential vectors (perpendicular check)
    perpendicularity = np.abs(np.einsum('ij,ij->i', normals, tangential_vectors))

    # Strongly reward normals that are perpendicular to the cone's surface
    scores = np.exp(-20 * (1 - perpendicularity) ** 2)
    return scores

def calculate_scores_based_on_perpendicularity_and_density(normals, points, apex, axis_direction, density_gradient):
    """
    Combines scores from perpendicularity checks and density gradient.

    Parameters:
        normals (ndarray): Nx3 array of normal vectors.
        points (ndarray): Nx3 array of 3D points.
        apex (ndarray): 3-element array representing the cone apex.
        axis_direction (ndarray): 3-element array representing the cone's axis direction.
        density_gradient (ndarray): Density gradient values indicating the progression from apex to base.

    Returns:
        ndarray: Combined score values for each point.
    """
    perpendicularity_scores = calculate_perpendicularity_scores(normals, points, apex, axis_direction)
    density_scores = 1 / (1 + np.exp(-10 * (density_gradient - 0.5)))

    # Combine scores to emphasize perpendicular normals and correct density progression
    combined_scores = perpendicularity_scores * density_scores
    return combined_scores

def ransac_cone_with_perpendicularity_scoring(points, normals, threshold, iterations, h_max, angle_min, angle_max):
    """
    RANSAC implementation to fit a cone, emphasizing perpendicularity of normals and density gradient.

    Parameters:
        points (ndarray): Nx3 array of 3D points.
        normals (ndarray): Nx3 array of normal vectors.
        threshold (float): Distance threshold for inlier determination.
        iterations (int): Number of iterations for RANSAC.
        h_max (float): Maximum allowed height constraint for the cone.
        angle_min (float): Minimum opening angle in degrees.
        angle_max (float): Maximum opening angle in degrees.

    Returns:
        tuple: Best-fit cone parameters (apex, axis_direction, opening_angle, fitted_height) and inliers.
    """
    angle_min_rad = np.deg2rad(angle_min)
    angle_max_rad = np.deg2rad(angle_max)
    
    best_inliers = []
    best_params = None
    best_score = 0

    for _ in range(iterations):
        idx = np.random.choice(points.shape[0], 3, replace=False)
        sample_points = points[idx]
        sample_normals = normals[idx]

        # Fit the cone from sampled points and normals
        apex, axis_direction, opening_angle = fit_cone_from_points_normals(sample_points, sample_normals)

        # Skip iteration if fitting fails or angle constraints are not met
        if (apex is None or axis_direction is None or opening_angle is None or 
            not (angle_min_rad <= opening_angle <= angle_max_rad)):
            continue

        # Calculate vectors from apex to all points and project onto the cone axis
        vectors_to_points = points - apex
        projection_length = np.dot(vectors_to_points, axis_direction)

        # Calculate the actual height of the fitted cone
        fitted_height = np.max(projection_length) - np.min(projection_length)

        # Enforce the height constraint (h_max)
        if fitted_height > h_max:
            continue

        # Calculate radial distances and expected distances based on the cone's opening angle
        projection = projection_length[:, np.newaxis] * axis_direction
        radial_distance = np.linalg.norm(vectors_to_points - projection, axis=1)
        expected_distance = np.tan(opening_angle) * np.abs(projection_length)

        height_violations = (projection_length < 0) | (projection_length > fitted_height)
        residuals = np.abs(radial_distance - expected_distance)

        # Calculate density gradient based on the entire point cloud, not just inliers
        density_gradient = calculate_density_gradient(points, apex, axis_direction)

        # Calculate scores focusing on perpendicularity and density gradient
        scores = calculate_scores_based_on_perpendicularity_and_density(
            normals, points, apex, axis_direction, density_gradient
        )

        # Filter points based on residuals and the simplified scoring system
        inliers = [(point, score) for idx, (point, residual, score) 
                   in enumerate(zip(points, residuals, scores))
                   if residual < threshold and not height_violations[idx] and score > 0.5]

        total_score = sum(score for _, score in inliers)

        # Update best model if the new model has a higher score
        if total_score > best_score or (len(inliers) > len(best_inliers) and total_score > 0.7 * best_score):
            best_inliers = inliers
            best_params = (apex, axis_direction, opening_angle, fitted_height)
            best_score = total_score

    best_inlier_points = [inlier[0] for inlier in best_inliers]
    return best_params, np.array(best_inlier_points)

In [4]:
import numpy as np
from scipy.optimize import least_squares
import open3d as o3d  # Make sure Open3D is installed

def fit_cone(points):
    def residuals(params, points):
        x0, y0, z0, a, b, c, theta = params
        direction_vector = np.array([a, b, c])
        direction_vector /= np.linalg.norm(direction_vector)  # Normalize the direction vector
        
        # Vector from the cone vertex to the point
        vector_to_point = points - np.array([x0, y0, z0])
        
        # Projection of the vector onto the cone's axis
        projection_length = np.dot(vector_to_point, direction_vector)
        projection = projection_length[:, np.newaxis] * direction_vector
        
        # Perpendicular distance from the point to the axis of the cone
        perpendicular_distance = np.linalg.norm(vector_to_point - projection, axis=1)
        
        # Expected distance based on the cone's angle
        expected_distance = np.tan(theta) * projection_length
        
        # Residuals are the difference between the actual and expected distances
        return perpendicular_distance - expected_distance

    # Initial guess for the cone parameters
    x0, y0, z0 = np.mean(points, axis=0)
    direction = np.array([0, 0, 1])  # Initial guess: cone points upwards
    theta0 = np.pi / 6  # Initial guess: 30 degrees opening angle
    initial_guess = [x0, y0, z0, direction[0], direction[1], direction[2], theta0]

    result = least_squares(residuals, initial_guess, args=(points,))
    return result.x

def ransac_cone(points, threshold, iterations):
    best_inliers = []
    best_params = None

    for _ in range(iterations):
        sample = points[np.random.choice(points.shape[0], 5, replace=False)]
        params = fit_cone(sample)
        inliers = []

        for point in points:
            x0, y0, z0, a, b, c, theta = params
            direction_vector = np.array([a, b, c])
            direction_vector /= np.linalg.norm(direction_vector)
            
            vector_to_point = point - np.array([x0, y0, z0])
            projection_length = np.dot(vector_to_point, direction_vector)
            projection = projection_length * direction_vector
            perpendicular_distance = np.linalg.norm(vector_to_point - projection)
            expected_distance = np.tan(theta) * projection_length
            
            distance = np.abs(perpendicular_distance - expected_distance)
            if distance < threshold:
                inliers.append(point)

        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            best_params = params

    return best_params, np.array(best_inliers)

In [5]:


cone_params, cone_inliers =  ransac_cone(points, threshold=0.01, iterations=1000)




In [6]:
plane_cloud = o3d.geometry.PointCloud()
plane_cloud.points = o3d.utility.Vector3dVector(cone_inliers)
plane_cloud.paint_uniform_color([0,1,0])

#sphere_cloud = o3d.geometry.PointCloud()
#sphere_cloud.points = o3d.utility.Vector3dVector(sphere_inliers)
#sphere_cloud.paint_uniform_color([0,0,1])

#cone_cloud = o3d.geometry.PointCloud()
#cone_cloud.points = o3d.utility.Vector3dVector(cone_inliers)
#cone_cloud.paint_uniform_color([1,0,0])

cone_mesh = create_cone_mesh(cone_params)
cone_mesh.paint_uniform_color([0.7, 1, 0.7])

downpcd.paint_uniform_color([0,0,0])

PointCloud with 4084 points.

## 3D Mesh

In [3]:

def create_cone_mesh(cone_params, resolution=50):
    """
    Create a cone mesh given cone parameters.

    Parameters:
    - cone_params: List containing [x0, y0, z0, a, b, c, theta]
    - resolution: Number of segments around the cone's base.

    Returns:
    - mesh: Open3D TriangleMesh object representing the cone.
    """
    x0, y0, z0, a, b, c, theta = cone_params
    height = 1.0  # Set the height of the cone; adjust as needed
    radius = height * np.tan(theta)

    # Create the base circle of the cone
    u = np.linspace(0, 2 * np.pi, resolution)
    base_x = radius * np.cos(u)
    base_y = radius * np.sin(u)
    base_z = np.zeros_like(base_x)

    # Create the apex of the cone
    apex = np.array([0, 0, height])

    # Combine vertices: base and apex
    vertices = np.vstack((np.column_stack((base_x, base_y, base_z)), apex))

    # Create triangles connecting the base to the apex
    triangles = [[i, (i + 1) % resolution, resolution] for i in range(resolution)]

    # Create the base triangles (optional; can leave base open)
    for i in range(1, resolution - 1):
        triangles.append([0, i, i + 1])

    # Create the mesh using Open3D
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(vertices)
    mesh.triangles = o3d.utility.Vector3iVector(triangles)
    mesh.compute_vertex_normals()

    # Transform the mesh to match the cone parameters
    mesh.translate([x0, y0, z0])

    return mesh

In [9]:
def create_cone_mesh(cone_params, height=1.0, resolution=50):
    """
    Create a cone mesh given cone parameters.

    Parameters:
    - cone_params: List containing [x0, y0, z0, a, b, c, theta]
    - height: Height of the cone (optional; adjust as needed).
    - resolution: Number of segments around the cone circumference.

    Returns:
    - mesh: Open3D TriangleMesh object representing the cone.
    """
    x0, y0, z0, a, b, c, theta = cone_params
    direction_vector = np.array([a, b, c])
    direction_vector /= np.linalg.norm(direction_vector)  # Normalize the direction vector

    # Create the base circle of the cone in the XY plane
    radius = height * np.tan(theta)
    circle_theta = np.linspace(0, 2 * np.pi, resolution)
    base_x = radius * np.cos(circle_theta)
    base_y = radius * np.sin(circle_theta)
    base_z = np.zeros_like(base_x)

    # Stack points to form the base circle and apex
    base_circle = np.column_stack((base_x, base_y, base_z))
    apex = np.array([0, 0, height])

    # Transform the base circle and apex to align with the cone's direction and position
    transform_matrix = np.eye(4)
    transform_matrix[:3, 3] = [x0, y0, z0]  # Translate to the cone's position

    # Rotation matrix to align with the direction vector
    axis = np.cross([0, 0, 1], direction_vector)
    if np.linalg.norm(axis) > 0:
        axis = axis / np.linalg.norm(axis)
        angle = np.arccos(np.dot([0, 0, 1], direction_vector))
        cos_angle = np.cos(angle)
        sin_angle = np.sin(angle)
        ux, uy, uz = axis
        rotation_matrix = np.array([
            [cos_angle + ux**2 * (1 - cos_angle), ux * uy * (1 - cos_angle) - uz * sin_angle, ux * uz * (1 - cos_angle) + uy * sin_angle],
            [uy * ux * (1 - cos_angle) + uz * sin_angle, cos_angle + uy**2 * (1 - cos_angle), uy * uz * (1 - cos_angle) - ux * sin_angle],
            [uz * ux * (1 - cos_angle) - uy * sin_angle, uz * uy * (1 - cos_angle) + ux * sin_angle, cos_angle + uz**2 * (1 - cos_angle)]
        ])
        base_circle = base_circle @ rotation_matrix.T
        apex = apex @ rotation_matrix.T

    base_circle += np.array([x0, y0, z0])
    apex = np.array([x0, y0, z0]) + apex

    # Combine vertices
    vertices = np.vstack((base_circle, apex))

    # Create triangles for the cone sides
    triangles = []
    for i in range(resolution - 1):
        # Side triangles connecting the base to the apex
        triangles.append([i, i + 1, resolution])

    # Close the loop at the end
    triangles.append([resolution - 1, 0, resolution])

    # Triangles for the base
    for i in range(1, resolution - 1):
        triangles.append([0, i, i + 1])

    # Create Open3D mesh
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(vertices)
    mesh.triangles = o3d.utility.Vector3iVector(triangles)
    mesh.compute_vertex_normals()

    return mesh


In [8]:
def create_sphere_mesh(sphere_params, resolution=50):
    center_x, center_y, center_z, radius = sphere_params
    u = np.linspace(0, 2 * np.pi, resoultion)
    v = np.linspace(0, np.pi, resolution)
    x = center_x + radius *np.outer(np.cos(u), np.sin(v))
    y = center_y + radius * np.outer(np.sin(u), np.sin(v))
    z = center_z + radius *np.outer(np.ones_like(u), np.cos(v))

    vertices = np.column_stack((x.ravel(), y.ravel(), z.ravel()))
    triangels = []
    for i in range(resolution-1):
        for j in range(resolution-1):
            v0 = i * resolution + j
            v1 = v0 + 1
            v2 = (i +1) * resolution + j
            v3 = v2 + 1
            triangles.extend([[v0, v2, v1], [v1, v2, v3]])

    mesh = 03d.geometry.TriangleMesh()
    mesh.vertices = 03d.utility.Vector3dVector(vertices)
    mesh.triangles = 03d.utility.Vector3iVector(triangles)
    mesh.compute_vertex_normals()'

    return mesh


In [10]:
o3d.visualization.draw_geometries([downpcd, plane_cloud, cone_mesh])